## Playground for HPC code


# I. Denoising

### 1. Models

In [1]:
import torch
import torch.nn as nn

class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        
        # Encoder: Downsamples the input image
        # Starting input shape: (batch_size, 3, 256, 256)
        self.encoder = nn.Sequential(
            # Convolution layer:
            #   - Converts 3 input channels to 64 channels.
            #   - Kernel size = 3, stride = 2, and padding = 1.
            #   - Output shape calculation: floor((256 - 3 + 2*1) / 2) + 1 = 128.
            #   - Resulting shape: (batch_size, 64, 128, 128)
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=2, padding=1),
            # Batch Normalization:
            #   - Normalizes the 64 feature maps.
            #   - Shape remains (batch_size, 64, 128, 128)
            nn.BatchNorm2d(64),
            # ReLU Activation:
            #   - Applies non-linearity (in-place for memory efficiency).
            #   - Shape remains (batch_size, 64, 128, 128)
            nn.ReLU(inplace=True),
            
            # Second convolution layer:
            #   - Converts 64 channels to 128 channels.
            #   - Same parameters: kernel size = 3, stride = 2, padding = 1.
            #   - Output shape: (batch_size, 128, 64, 64) [since 128/2 = 64]
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2, padding=1),
            # Batch Normalization:
            #   - Normalizes the 128 feature maps.
            #   - Shape remains (batch_size, 128, 64, 64)
            nn.BatchNorm2d(128),
            # ReLU Activation:
            #   - Shape remains (batch_size, 128, 64, 64)
            nn.ReLU(inplace=True),
            
            # Third convolution layer:
            #   - Converts 128 channels to 256 channels.
            #   - Same parameters: kernel size = 3, stride = 2, padding = 1.
            #   - Output shape: (batch_size, 256, 32, 32) [since 64/2 = 32]
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=2, padding=1),
            # Batch Normalization:
            #   - Normalizes the 256 feature maps.
            #   - Shape remains (batch_size, 256, 32, 32)
            nn.BatchNorm2d(256),
            # ReLU Activation:
            #   - Shape remains (batch_size, 256, 32, 32)
            nn.ReLU(inplace=True)
        )
        
        # Decoder: Upsamples to reconstruct the image
        # Starting input shape for decoder: (batch_size, 256, 32, 32)
        self.decoder = nn.Sequential(
            # First transposed convolution (upsampling) layer:
            #   - Converts 256 channels to 128 channels.
            #   - Kernel size = 3, stride = 2, padding = 1, and output_padding = 1.
            #   - Output size calculation for ConvTranspose2d: (32 - 1)*2 - 2*1 + 3 + 1 = 64.
            #   - Resulting shape: (batch_size, 128, 64, 64)
            nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=3, stride=2, padding=1, output_padding=1),
            # Batch Normalization:
            #   - Normalizes the 128 feature maps.
            #   - Shape remains (batch_size, 128, 64, 64)
            nn.BatchNorm2d(128),
            # ReLU Activation:
            #   - Shape remains (batch_size, 128, 64, 64)
            nn.ReLU(inplace=True),
            
            # Second transposed convolution layer:
            #   - Converts 128 channels to 64 channels.
            #   - Same parameters yield output shape: (batch_size, 64, 128, 128)
            nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=2, padding=1, output_padding=1),
            # Batch Normalization:
            #   - Normalizes the 64 feature maps.
            #   - Shape remains (batch_size, 64, 128, 128)
            nn.BatchNorm2d(64),
            # ReLU Activation:
            #   - Shape remains (batch_size, 64, 128, 128)
            nn.ReLU(inplace=True),
            
            # Third transposed convolution layer:
            #   - Converts 64 channels back to 3 channels (reconstructing the RGB image).
            #   - Parameters yield output shape: (batch_size, 3, 256, 256)
            nn.ConvTranspose2d(in_channels=64, out_channels=3, kernel_size=3, stride=2, padding=1, output_padding=1),
            # Sigmoid Activation:
            #   - Squashes the output values to range [0, 1] (useful if your image pixel values are normalized).
            nn.Sigmoid()
        )
        
    def forward(self, x):
        # x: Input tensor of shape (batch_size, 3, 256, 256)
        encoded = self.encoder(x)   # Encoded output shape: (batch_size, 256, 32, 32)
        print("Encoded shape:", encoded.shape)
        decoded = self.decoder(encoded)  # Reconstructed output shape: (batch_size, 3, 256, 256)
        return decoded
    # Example usage:
if __name__ == '__main__':
    # Create the model
    model = ConvAutoencoder()
    # print(model)
    
    # Create a random sample input tensor (e.g., batch of 1 image of size 256x256 with 3 channels)
    sample_input = torch.randn(1, 3, 256, 256)
    
    # Forward pass through the network
    output = model(sample_input)
    print("Output shape:", output.shape)

Encoded shape: torch.Size([1, 256, 32, 32])
Output shape: torch.Size([1, 3, 256, 256])


Summary of Shape Transformations

	1.	Encoder:
	•	Input: (batch_size, 3, 256, 256)
	•	After 1st Conv: (batch_size, 64, 128, 128)
	•	After 2nd Conv: (batch_size, 128, 64, 64)
	•	After 3rd Conv: (batch_size, 256, 32, 32)
	
	2.	Decoder:
	•	Input: (batch_size, 256, 32, 32)
	•	After 1st ConvTranspose: (batch_size, 128, 64, 64)
	•	After 2nd ConvTranspose: (batch_size, 64, 128, 128)
	•	After 3rd ConvTranspose: (batch_size, 3, 256, 256)


In [ ]:
# Conv2d and ConvTranspose2d


In [4]:
# Torch
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch import optim 


# Train + Data 
import sys 
sys.path.append('../Layers')
from Conv1d_NN import * 
from Conv2d_NN import * 
from Conv1d_NN_spatial import *
from Conv2d_NN_spatial import * 

sys.path.append('../Data')
from CIFAR10 import * 


sys.path.append('../Train')
from train2d import * 

/Users/mingikang/miniforge3/envs/ML/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# ConvNN


class Denoising_AutoEncoder_ConvNN_K_All(nn.Module):
    def __init__(self, 
                in_channels, 
                out_channels, 
                K=3,
                stride=3, 
                padding=0, 
                shuffle_pattern="B", 
                shuffle_scale=2, 
                samples="all", 
                magnitude_type="similarity",
                location_channels=False
            ):
        super(Denoising_AutoEncoder_ConvNN_K_All, self).__init__() 
        
        assert K == stride, "K and stride must be equal"
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.K = K
        self.stride = stride
        self.padding = padding
        self.shuffle_pattern = shuffle_pattern  
        self.shuffle_scale = shuffle_scale
        self.samples = samples
        self.magnitude_type = magnitude_type
        self.location_channels = location_channels
        
        self.name = "Denoising_AutoEncoder_ConvNN_K_All"
        
        
        
        self.encoder = nn.Sequential(
            Conv2d_NN(in_channels=self.in_channels, out_channels=64, K=self.K, stride=self.stride, padding=self.padding, shuffle_pattern=self.shuffle_pattern),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True), 
            Conv2d_NN(in_channels=64, out_channels=128, K=self.K, stride=self.stride, padding=self.padding, shuffle_pattern=self.shuffle_pattern),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            Conv2d_NN(in_channels=128, out_channels=256, K=self.K, stride=self.stride, padding=self.padding, shuffle_pattern=self.shuffle_pattern),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True)
        )
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=self.K, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=self.K, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels=64, out_channels=self.out_channels, kernel_size=self.K, stride=self.stride, padding=self.padding),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        print("Input shape:", x.shape)
        print("Starting encoder layer")
        for layer in self.encoder:
            x = layer(x)
            print(f"- ", x.shape)
        print("After encoder shape:", x.shape)
        
        print("Starting decoder layer")
        for layer in self.decoder:
            x = layer(x)
            print(f"- ", x.shape)
        print("After decoder shape:", x.shape)
            
        return x

    

# Example usage:
if __name__ == '__main__':
    # Create the model
    model = Denoising_AutoEncoder_ConvNN_K_All(in_channels=3, out_channels=3)
    # print(model)
    
    # Create a random sample input tensor (e.g., batch of 1 image of size 256x256 with 3 channels)
    sample_input = torch.randn(1, 3, 256, 256)
    
    # Forward pass through the network
    output = model(sample_input)
    print("Output shape:", output.shape)

Input shape: torch.Size([1, 3, 256, 256])
Starting encoder layer
-  torch.Size([1, 256, 128, 128])


RuntimeError: running_mean should contain 256 elements not 64

## II. Classification

In [8]:
import torch
import torch.nn as nn

class SimpleCNNClassifier(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNNClassifier, self).__init__()
        
        # Convolutional Block 1:
        # Input shape: (batch_size, 3, 224, 224)
        self.layer1 = nn.Sequential(
            # Conv2d:
            #   - Converts 3 channels to 32 channels.
            #   - Kernel size = 3, stride = 1, and padding = 1 keeps spatial dims same (224x224).
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1),
            # Batch Normalization:
            #   - Normalizes the 32 feature maps.
            nn.BatchNorm2d(32),
            # ReLU Activation:
            #   - Applies non-linearity in-place.
            nn.ReLU(inplace=True),
            # Max Pooling:
            #   - Reduces spatial dimensions by half: 224 -> 112.
            nn.MaxPool2d(kernel_size=2, stride=2)
            # Output shape: (batch_size, 32, 112, 112)
        )
        
        # Convolutional Block 2:
        # Input shape: (batch_size, 32, 112, 112)
        self.layer2 = nn.Sequential(
            # Conv2d:
            #   - Converts 32 channels to 64 channels.
            #   - With same kernel parameters, spatial dims remain 112 x 112.
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            # Batch Normalization:
            nn.BatchNorm2d(64),
            # ReLU Activation:
            nn.ReLU(inplace=True),
            # Max Pooling:
            #   - Further halves spatial dimensions: 112 -> 56.
            nn.MaxPool2d(kernel_size=2, stride=2)
            # Output shape: (batch_size, 64, 56, 56)
        )
        
        # Convolutional Block 3:
        # Input shape: (batch_size, 64, 56, 56)
        self.layer3 = nn.Sequential(
            # Conv2d:
            #   - Converts 64 channels to 128 channels.
            #   - Keeps spatial dims at 56 x 56 with padding.
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            # Batch Normalization:
            nn.BatchNorm2d(128),
            # ReLU Activation:
            nn.ReLU(inplace=True),
            # Max Pooling:
            #   - Halves spatial dimensions: 56 -> 28.
            nn.MaxPool2d(kernel_size=2, stride=2)
            # Output shape: (batch_size, 128, 28, 28)
        )
        
        # Fully Connected Layers:
        # Flatten the output from the convolutional blocks.
        # The flattened size is 128 channels * 28 * 28 = 100352 features per sample.
        self.fc = nn.Sequential(
            # First fully connected layer:
            nn.Linear(128 * 28 * 28, 256),
            nn.ReLU(inplace=True),
            # Dropout for regularization:
            nn.Dropout(0.5),
            # Final fully connected layer for classification:
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        # x: Input tensor with shape (batch_size, 3, 224, 224)
        out = self.layer1(x)   # -> (batch_size, 32, 112, 112)
        out = self.layer2(out) # -> (batch_size, 64, 56, 56)
        out = self.layer3(out) # -> (batch_size, 128, 28, 28)
        # Flatten the feature maps into a vector: (batch_size, 128*28*28)
        out = out.view(out.size(0), -1)
        # Fully connected layers output class scores: (batch_size, num_classes)
        out = self.fc(out)
        return out

# Example usage:
if __name__ == '__main__':
    model = SimpleCNNClassifier(num_classes=10)
    print(model)
    
    # Create a sample input tensor: a batch with one image (3, 224, 224)
    sample_input = torch.randn(1, 3, 224, 224)
    # Forward pass through the network
    output = model(sample_input)
    print("Output shape:", output.shape)  # Expected: (1, 10)

SimpleCNNClassifier(
  (layer1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=100352, out_featur